In [43]:
import pandas as pd
dataset = pd.read_csv("flowWithSelfish.csv")
dataset.drop(dataset.columns[0], axis=1, inplace=True)
dataset.head()
# dataset.columns

,Source,Destination,Protocol,Src_port,Dest_port,Flow_count,Flow_size,pkt_size_of_first_flow,Flow_duration,first_flow_inter_arrival_time,pkt_size_to_flow_size,isSelfish
0,0.0.0.0,255.255.255.255,DHCP,-1,-1,279,95418,342,3335.924975,0.005128,0.003584,0
1,1.1.247.102,103.119.242.114,TCP,-1,-1,5,330,66,293.814945,0.135032,0.200000,0
2,1.1.247.102,103.119.242.114,TCP,4001,4001,1,66,66,254.990643,0.747853,1.000000,0
3,1.156.246.187,103.119.242.114,UDP,37186,32478,1,331,331,1821.708214,0.227403,1.000000,0
4,1.164.136.162,103.119.242.114,UDP,7536,32478,1,143,143,950.648788,3.748469,1.000000,0


In [44]:
X = dataset.iloc[:, 3:11].values
Y = dataset.iloc[:, 11].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [45]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=1.0, solver="lbfgs", multi_class="ovr")
from sklearn.feature_selection import RFE
selector = RFE(classifier, n_features_to_select=0.8)
selector = selector.fit(X_train, y_train)
order = []
for i in range(0, selector.support_.size) : 
    if(selector.support_[i]) :
        order.append(i+3)
order
        

[3, 5, 6, 8, 9, 10]

In [46]:
for i in order :
    print(i, f"{dataset.columns[i]}")

3 Src_port
5 Flow_count
6 Flow_size
8 Flow_duration
9 first_flow_inter_arrival_time
10 pkt_size_to_flow_size


In [47]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[2109    0]
 [ 228    0]]
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2109
           1       0.00      0.00      0.00       228

    accuracy                           0.90      2337
   macro avg       0.45      0.50      0.47      2337
weighted avg       0.81      0.90      0.86      2337

0.9024390243902439
